# OSP model



### Import libraries

In [ ]:
%load_ext lab_black
import meta, evaluate, h5py
import numpy as np
import matplotlib.pyplot as plt

### Load model configuration

In [ ]:
cfg = meta.model_cfg("models/O2P_replication_set11_r0042/model_config.json")
vis = evaluate.vis(cfg.path_model_folder)

### Training history

In [ ]:
vis.training_hist()

### Strain plots

In [ ]:
vis.plot_dev_interactive("acc", ["strain", "grain"]).properties(
    title="Accuracy in Strain and Grain"
)

### Grain plots

In [ ]:
small = vis.plot_dev_interactive("acc_small_grain", exp=["grain"]).properties(
    title="Small Grain Response"
)
large = vis.plot_dev_interactive("acc_large_grain", exp=["grain"]).properties(
    title="Large Grain Response"
)

(small | large).properties(title="Accuracy of Grain by response and condition")

### Taraban plots

In [ ]:
vis.plot_dev_interactive("acc", ["taraban"]).properties(
    title="Accuracy in Taraban by condition"
)

### Glushko plots

In [ ]:
vis.plot_dev_interactive("acc", ["glushko"]).properties(
    title="Accuracy in Glushko by condition"
)

### Words vs. Nonwords

In [ ]:
wnw_zr = vis.plot_wnw(["INC_HF"], ["unambiguous"]).properties(
    title="Strain (INC_HF) vs. Grain (UN)"
)

all_taraban_conds = list(vis.cdf.loc[vis.cdf.exp == "taraban", "cond"].unique())
wnw_tg = vis.plot_wnw(all_taraban_conds, ["Exception", "Regular"]).properties(
    title="Taraban (all) vs. Glushko (NW)"
)
wnw_zr | wnw_tg

### Weights

In [ ]:
vis.weight.violinplot(cfg.path_plot_folder + "weight_violinplot.png")

In [ ]:
ax = vis.weight.df.boxplot(
    column="abs_weight", by="matrix", grid=False, figsize=(10, 5)
)

In [ ]:
vis.weight.boxplot(cfg.path_plot_folder + "weight_boxplot.png")

In [ ]:
vis.weight.basic_stat()

In [ ]:
!jupyter nbconvert --output-dir=$cfg.path_model_folder --to html OSP_master.ipynb